In [2]:
!pip install websocket_client
!pip install python-binance

     |████████████████████████████████| 53 kB 1.3 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 103 kB 11.7 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 288 kB 58.3 MB/s 
     |████████████████████████████████| 1.1 MB 50.2 MB/s 
     |████████████████████████████████| 144 kB 53.3 MB/s 
     |████████████████████████████████| 271 kB 43.8 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 


In [19]:
import websocket, json
from binance.exceptions import BinanceAPIException
from binance.client import Client
from binance.enums import *

SYMBOL = "busdusdt"
SOCKET = "wss://stream.binance.com:9443/ws/"+SYMBOL+"@kline_1m"



In [ ]:
client = Client(API_KEY, SECRET_KEY)

In [24]:
balances = client.get_account()['balances']

for b in balances[0:200]:
  if b['asset'] in ['BUSD', 'USDT']:
    print(b)

{'asset': 'USDT', 'free': '6.36005384', 'locked': '0.00000000'}
{'asset': 'BUSD', 'free': '20.00000000', 'locked': '0.00000000'}


In [31]:
balances = client.get_account()['balances']
print('After Trade')
for b in balances[0:200]:
  if b['asset'] in ['BUSD', 'USDT']:
    print(b)

After Trade
{'asset': 'USDT', 'free': '26.35005384', 'locked': '0.00000000'}
{'asset': 'BUSD', 'free': '0.00000000', 'locked': '0.00000000'}


In [26]:
def on_open(ws):
  print("Connction Opened")

def on_close(ws):
  print("Connction close")

def on_error(ws, error):
  print("Here is an error:", error)

def order(side, quantity, symbol="BUSDUSDT"):
  try:
    my_order = client.create_order(symbol= symbol, side = side, type= client.ORDER_TYPE_MARKET, quantity = quantity)
    print(my_order)
  except BinanceAPIException as e:
    print(e)


def on_message(ws, message):
  print("Message Recieved")
  candle_data = json.loads(message)
  closing_rate = float(candle_data['k']['c'])

  print("Closing:", closing_rate)

  if mode == "SELL" and closing_rate >= sell_limit:
    #sell the coins
    print("SELLLL ORDER")
    order(mode, amount)
    if repeat == 'Once':
      ws.close()
  elif mode == "BUY" and closing_rate <= buy_limit:
    print("BUYYYYYYYYYYY ORDER")
    order(mode, amount)
    if repeat == 'Once':
      ws.close()
    #buy coins

  print("Closing:", closing_rate)

In [29]:
repeat = 'Once' #@param ['Once', 'Keep Repeating']
mode = 'BUY' #@param ['BUY', 'SELL']
amount =  20#@param {type:"number"}
sell_limit =  0.9996#@param {type:"number"}
buy_limit =  0.9995#@param {type:"number"}


In [30]:
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_error=on_error, on_message=on_message)
ws.run_forever()

Connction Opened
Message Recieved
Closing: 0.9995
Closing: 0.9995
Message Recieved
Closing: 0.9995
Closing: 0.9995
Message Recieved
Closing: 0.9996
SELLLL ORDER
{'symbol': 'BUSDUSDT', 'orderId': 401484506, 'orderListId': -1, 'clientOrderId': 'P25O8AZvxadO0m9DA614PV', 'transactTime': 1643721955387, 'price': '0.00000000', 'origQty': '20.00000000', 'executedQty': '20.00000000', 'cummulativeQuoteQty': '19.99000000', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'SELL', 'fills': [{'price': '0.99950000', 'qty': '20.00000000', 'commission': '0.00000000', 'commissionAsset': 'USDT', 'tradeId': 248858427}]}
Closing: 0.9996
